In [1]:
import pandas as pd

# Load dataset
data = pd.read_csv('../UpdatedFilteredUsedData.csv')
data.head()

C:\Users\Gunne\AppData\Local\Temp\ipykernel_39868\1480144099.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../UpdatedFilteredUsedData.csv')


,Year,Make,Model,Price,Mileage,Location,Additional,Fuel Type,Car Type,Scraping Time,Source
0,2021,Audi,RS7,214880,21601,NSW,*Excl. Govt. Charges,Petrol,hatch,1712976689,Drive
1,2021,Audi,RS7,214880,21601,NSW,*Excl. Govt. Charges,Petrol,hatch,1713581171,Drive
2,2021,Audi,RS7,214880,21601,NSW,*Excl. Govt. Charges,Petrol,hatch,1714183161,Drive
3,2021,Audi,RS7,214880,21601,NSW,*Excl. Govt. Charges,Petrol,hatch,1714787697,Drive
4,2021,Audi,RS7,214880,21601,NSW,*Excl. Govt. Charges,Petrol,hatch,1715394085,Drive


In [2]:
# Define the columns to check for duplicates
columns_to_check = ['Year', 'Make', 'Model', 'Price', 'Mileage', 'Location']

before_removal = len(data)

# Drop duplicates based on the specified columns
data_unique = data.drop_duplicates(subset=columns_to_check)

after_removal = len(data_unique)

removed = before_removal - after_removal
removed, after_removal

(622691, 346936)

In [3]:
# Number of listings per year for each fuel
data = data_unique

data['Fuel Type Adjusted'] = data.apply(lambda x:
                                        'Electric (Tesla)' if 'Tesla' in x['Make'] and x['Fuel Type'] == 'Electric' else
                                        ('Electric (Non-Tesla)' if x['Fuel Type'] == 'Electric' else
                                         ('Hybrid' if x['Fuel Type'] == 'Hybrid' else 'Conventional')),
                                        axis=1)

# Group by 'Year' and 'Fuel Type Adjusted' and count the number of listings
listings_by_year_fuel = data.groupby(['Year', 'Fuel Type Adjusted']).size().reset_index(name='Count')

listings_by_year_fuel

,Year,Fuel Type Adjusted,Count
0,2015,Conventional,25184
1,2015,Electric (Non-Tesla),28
2,2015,Electric (Tesla),37
3,2015,Hybrid,253
4,2016,Conventional,29121
5,2016,Electric (Non-Tesla),20
6,2016,Electric (Tesla),8
7,2016,Hybrid,352
8,2017,Conventional,35423
9,2017,Electric (Non-Tesla),26


In [4]:
# Function to compute statistics of df
def compute_statistics(df):
    # Group by the adjusted fuel type
    grouped = df.groupby('Fuel Type Adjusted')

    # Calculate statistics
    stats = pd.DataFrame({
        'Number Listed': grouped.size(),
        'Number of Different Makes': grouped['Make'].nunique(),
        'Mean Mileage': grouped['Mileage'].mean(),
        'Std Mileage': grouped['Mileage'].std(),
        'Median Mileage': grouped['Mileage'].median(),
        'Mean Age': grouped['Age'].mean(),
        'Std of Age': grouped['Age'].std(),
        'Median Age': grouped['Age'].median(),
        'Mean Price': grouped['Price'].mean(),
        'Std Price': grouped['Price'].std(),
        'Median Price': grouped['Price'].median(),
    })

    return stats

# Current year to calculate the age of the vehicles
current_year = pd.Timestamp.now().year

# Add 'Age' column
data['Age'] = current_year - data['Year']

# Compute statistics for all vehicles
all_vehicle_stats = compute_statistics(data)

# Compute statistics for SUVs specifically
suv_stats = compute_statistics(data[data['Car Type'] == 'suv'])

print("All Vehicle Statistics by Fuel Type:")
print(all_vehicle_stats)

print("\nSUV Statistics by Fuel Type:")
print(suv_stats)


All Vehicle Statistics by Fuel Type:
                      Number Listed  Number of Different Makes  Mean Mileage  \
Fuel Type Adjusted                                                             
Conventional                 320808                         75  74635.362669   
Electric (Non-Tesla)           3987                         29  14489.634813   
Electric (Tesla)               1020                          1  32142.048039   
Hybrid                        21121                         27  48147.460774   

                       Std Mileage  Median Mileage  Mean Age  Std of Age  \
Fuel Type Adjusted                                                         
Conventional          60318.781237         66098.0  4.830774    2.434999   
Electric (Non-Tesla)  18796.335830          6401.0  2.002508    1.563506   
Electric (Tesla)      21659.864336         27035.0  3.013725    1.935619   
Hybrid                32588.058745         46810.0  3.306756    1.771587   

                      Med